## Import Statements

In [ ]:
from openai import AzureOpenAI
from dotenv import load_dotenv 
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True) # take environment variables from .env.

## Make an call to Azure OpenAI without a data source using an API kep

In [ ]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")
deployment_name = os.getenv("AZURE_OPENAI_MODEL")

client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version="2024-05-01-preview",
)

completion = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assisant.",
        },
        {
            "role": "user",
            "content": "tell me about the current Aquatic Education Programs"
        },
    ]
)

print(completion.choices[0].message.content)

In [ ]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
print(azure_endpoint)

api_key=os.getenv("AZURE_OPENAI_KEY")


In [ ]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")
deployment_name = os.getenv("AZURE_OPENAI_MODEL")

client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version="2024-05-01-preview",
)

completion = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assisant.",
        },
        {
            "role": "user",
            "content": "tell me about the current Aquatic Education Programs"
        },
    ]
)

print(completion.choices[0].message.content)

In [ ]:
import os
#from azure.ai.openai import AzureOpenAI
from retry import retry  # Ensure to install this library: pip install retry

# Retrieve environment variables
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_KEY")
deployment_name = os.getenv("AZURE_OPENAI_MODEL")

# Initialize Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version="2024-05-01-preview",
)

# Define a retry decorator for handling transient errors
@retry(tries=3, delay=2, backoff=2, exceptions=(TimeoutError, ConnectionError))
def get_completion_with_retry():
    # Call the chat completions API with a timeout
    return client.chat.completions.create(
        model=deployment_name,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": "tell me about the current Aquatic Education Programs"
            },
        ],
        timeout=45  # Timeout in seconds
    )

# Execute the API call with retry
try:
    completion = get_completion_with_retry()
    print(completion.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {e}")


## Ask a question to Azure OpenAI with a Datasource

In [ ]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")
deployment_name = os.getenv("AZURE_OPENAI_MODEL")


client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version="2024-05-01-preview",
)

completion = client.chat.completions.create(
    model=deployment_name,
    messages=[
         {
            "role": "system",
            "content": "You are a helpful assisant.",
        },
        {
            "role": "user",
            "content": "tell me about the current Aquatic Education Programs",
        },
    ],
    extra_body={
        "data_sources":[
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"],
                    "index_name": os.environ["AZURE_SEARCH_INDEX"],
                    "authentication": {
                        "type": "api_key",
                        "key": os.environ["AZURE_SEARCH_API_KEY"],
                    }
                }
            }
        ],
    }
)

print(completion.choices[0].message.content)



## Azure OpenAI with Token Provider & Managed Identity with a data soruce

- Format code

In [ ]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_MODEL")
system_message = "you are a helpful AI Assistant"

token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    azure_ad_token_provider=token_provider,
    api_version="2024-05-01-preview",
)

completion = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {
            "role": "user",
            "content": "tell me about the current Aquatic Education Programs",
        },
    ],
    extra_body={
                    "data_sources": [
                        {
                            "type": "azure_search",
                            "parameters": {
                                "authentication": {
                                    "type": "system_assigned_managed_identity"
                                    },
                                "endpoint": os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"],
                                "index_name": os.environ["AZURE_SEARCH_INDEX"],
                                "fields_mapping": {
                                    "content_fields": ["chunk"],
                                    "vector_fields": ["vector"],
                                    "title_field": "title",
                                    "url_field": "chunk_id",
                                    "filepath_field": "page_number",

                                },
                                "filter": None,
                                "in_scope": True,
                                "strictness": 2,
                                "top_n_documents": 5,
                                "embedding_dependency": {
                                    "type": "deployment_name",
                                    "deployment_name": os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME"),
                                },
                                "query_type": "vector_semantic_hybrid",
                                "semantic_configuration": os.getenv("AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG"),
                                "role_information": system_message,
                            },
                        }
                    ]
                }
)

print(completion.choices[0].message.content)



content = completion.choices[0].message.content

context = completion.choices[0].message.context
for citation_index, citation in enumerate(context["citations"]):
    citation_reference = f"[doc{citation_index + 1}]"
    filepath = citation["filepath"]
    title = citation["title"]
    snippet = citation["content"]
    chunk_id = citation["chunk_id"] 
    url = "https://blah" + title + ":" + filepath # replace with actual host and encode the URL
    replaced_html = f"<a href='{url}'</a>"
    content = content.replace(citation_reference, replaced_html)

print('############################')
print(content)

print(chunk_id + content)




